In [11]:
import cv2
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from glob import glob

In [12]:
COLOR_TRAIN_DIR = "data/train_color"
BLACK_TRAIN_DIR = "data/train_black"
COLOR_TEST_DIR = "data/test_color"
BLACK_TEST_DIR = "data/test_black2"
IMG_SIZE = 128 #images are 400x400 ---- Make them smaller to use less memory but keep accuracy
BATCH_SIZE = 16 # hyperparameter we need to optimize
EPOCHS = 5
NUMBER_OF_IMAGES = 500
DEVICE = torch.device("cpu") #running on my macbook without a gpu ---- CAN ADD GPU SUPPPORT FOR OTHER DEVICES

## Step 1 is to create a class that can hold our loaded datasets, as well as process the images in our dataset for the model

In [13]:
class ColorizationDataset(Dataset):
    # Built in double underscore python method to be the classe's constructor
    def __init__(self, colorPaths, grayscalePaths, imageSize):
        # This will define all images to class variables that are passed in the paths. i.e. "data/train_color" for self.colorpaths
        self.colorPaths = sorted(glob(os.path.join(colorPaths, '*.jpg')))[:NUMBER_OF_IMAGES]
        self.grayscalePaths = sorted(glob(os.path.join(grayscalePaths, '*.jpg')))[:NUMBER_OF_IMAGES]
        self.imageSize = imageSize

    # Built in double underscore python method to get length of data that our class instance holds
    def __len__(self):
        return len(self.grayscalePaths)

    # Function to process grayscale training photos into L color space channel and return tensor
    def processGrayscale(self, grayscaleImagePath):
        # IMAGE GETS READ IN BGR COLOR SPACE
        grayscaleImage = cv2.imread(grayscaleImagePath)
        grayscaleImage = cv2.resize(grayscaleImage, (self.imageSize, self.imageSize))

        # Defines LAB color space by using Opencv functions to take a colored image and put in into Lab
        labColorSpace = cv2.cvtColor(grayscaleImage, cv2.COLOR_BGR2Lab)
        
        # Gets the chrominance of a and b channels by making all values within the range [-1,1] (Commonly used in neural networks)
        L, a, b = cv2.split(labColorSpace)
        # Since IMREAD_GRAYSCALE is already in L channel, we can just normalize grayscaleimage to be within -1 to 1
        L = (L.astype(np.float32) - 128) / 128
        L = torch.tensor(L, dtype=torch.float32).unsqueeze(0)
        return L

    # Function to process color training photos into a and b channels into a single stacked tensor
    def processColor(self, colorImagePath):
        # IMAGE GETS READ IN BGR COLOR SPACE
        colorImage = cv2.imread(colorImagePath)
        colorImage = cv2.resize(colorImage, (self.imageSize, self.imageSize))
        
        # Defines LAB color space by using Opencv functions to take a colored image and put in into Lab
        labColorSpace = cv2.cvtColor(colorImage, cv2.COLOR_BGR2Lab)

        # Gets the chrominance of a and b channels by making all values within the range [-1,1] (Commonly used in neural networks)
        L, a, b = cv2.split(labColorSpace)
    
        # Normalize a and b values to be within -1 to 1
        L = (L.astype(np.float32) - 128) / 128
        a = (a.astype(np.float32) - 128) / 128
        b = (b.astype(np.float32) - 128) / 128
        # Using stack to speed-up performance by joining a and b channels stacked on the 0 axis (All of a stacked on all of b)
        lab = torch.tensor(np.stack([L,a,b], axis=0), dtype=torch.float32)

        return lab

    # Built in double underscore python method to get the index of the item in our class instance.
    def __getitem__(self, idx):
        grayscaleImagePath = self.grayscalePaths[idx]
        colorImagePath = self.colorPaths[idx]
        processedGrayscale = self.processGrayscale(grayscaleImagePath)
        processedColor = self.processColor(colorImagePath)
        return processedColor, processedGrayscale        

In [14]:

class ColorizationModelNeuralNetwork(nn.Module):
    # Class construction using same built in python double underscore method
    def __init__(self):
        super(ColorizationModelNeuralNetwork, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3,3), padding=1),
            nn.BatchNorm2d(64),  # Add BatchNorm
            nn.ReLU(), # USING THE RELU ACTIVATION FUNCTION LIKE WE SAW IN CLASS
            nn.Conv2d(64, 128, kernel_size=(3,3), padding=1),
            nn.BatchNorm2d(128),  # Add BatchNorm
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=(3,3), padding=1),
            nn.BatchNorm2d(256),  # Add BatchNorm
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=(3,3), padding=1),
            nn.BatchNorm2d(128),  # Add BatchNorm
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=(3,3), padding=1),
            nn.BatchNorm2d(64),  # Add BatchNorm
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=(3,3), padding=1),
            nn.Tanh()
        )


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [15]:
# This will put our images into a class dataset so we can load and process the images before training the model
train_dataset = ColorizationDataset(COLOR_TRAIN_DIR, BLACK_TRAIN_DIR, IMG_SIZE)
test_dataset = ColorizationDataset(COLOR_TEST_DIR, BLACK_TEST_DIR, IMG_SIZE)

# Loads the data correctly in a certain batch size to help with the EPOCH's in our training process.
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Now we want to initialize the model, the mean squared error loss criterion, and the optimizer for our model

In [16]:
colorizerModel = ColorizationModelNeuralNetwork().to(DEVICE)
criterion = nn.MSELoss()
optimizer = optim.Adam(colorizerModel.parameters(), lr=0.0001)

## Now Load Our Image Dataset we chose and Train the Model

In [17]:
import matplotlib.pyplot as plt

# Initialize lists for epoch and loss tracking
epoch_losses = []

for epoch in range(EPOCHS):
    colorizerModel.train()
    running_loss = 0.0

    for color_lab_space, grayScale_L_channel in train_loader:
        color_lab_space, grayScale_L_channel = color_lab_space.to(DEVICE), grayScale_L_channel.to(DEVICE)
        optimizer.zero_grad()
        grayscale_lab_prediction = colorizerModel(grayScale_L_channel)

        loss = criterion(grayscale_lab_prediction, color_lab_space)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)  # Average loss for this epoch
    epoch_losses.append(avg_loss)  # Store loss for plotting

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}")

# Plot after training
plt.figure(figsize=(8, 6))
plt.plot(range(1, EPOCHS + 1), epoch_losses, marker='o', label="Average Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.legend()
plt.grid(True)
plt.show()

Epoch 1/5, Loss: 0.0396
Epoch 2/5, Loss: 0.0162
Epoch 3/5, Loss: 0.0135
Epoch 4/5, Loss: 0.0128
Epoch 5/5, Loss: 0.0122


<Figure size 800x600 with 1 Axes>

In [18]:
def colorizeImage(model, imagePath):
    # IMAGE GETS READ IN BGR COLOR SPACE
    grayscaleTestImage = cv2.imread(imagePath)
    labColorSpaceImage = labColorSpace = cv2.cvtColor(grayscaleTestImage, cv2.COLOR_BGR2Lab)

    L_space, a, b = cv2.split(labColorSpaceImage)
    L_space = (L_space.astype(np.float32) - 128) / 128
    L_space_tensor = torch.tensor(L_space, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    L_space_tensor = L_space_tensor.to(DEVICE)
    print(L_space_tensor.shape)
    with torch.no_grad():
        lab_prediction = model(L_space_tensor).cpu()

    lab_prediction = lab_prediction.squeeze().numpy()
    # Un-regularizes the values that were between [-1,1]
    l = lab_prediction[0].astype(np.float32) * 128 + 128
    a = lab_prediction[1].astype(np.float32) * 128 + 128
    b = lab_prediction[2].astype(np.float32) * 128 + 128
    
    #L = ((L_image * 128) + 128) 
    
    # uint8 is so that the values will be 2^8 bit so range will be within rgb values
    colorized_lab = cv2.merge([l.astype(np.uint8), a.astype(np.uint8), b.astype(np.uint8)])

    colorized_rgb = cv2.cvtColor(colorized_lab, cv2.COLOR_LAB2RGB)

    plt.imshow(colorized_rgb)
    plt.axis('off')
    plt.show()

def colorizeTestSet(model, imageSetPath):
    for image in os.listdir(imageSetPath):
        if image.endswith('.jpg'):
            grayscalePath = os.path.join(imageSetPath, image)
            colorizeImage(model, grayscalePath)

colorizeTestSet(colorizerModel, BLACK_TEST_DIR)

torch.Size([1, 400, 400])


<Figure size 640x480 with 1 Axes>

torch.Size([1, 400, 400])


<Figure size 640x480 with 1 Axes>